In [ ]:
#| label: b0Fig7jn

# Prepare Python environment
import numpy as np
import plotly.express as px
import os
import nibabel as nib
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import json

import scipy.io as sio
from pathlib import Path
import os
import sys
from contextlib import contextmanager
from repo2data.repo2data import Repo2Data

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(__file__).parents[2]
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)

data_dir = dataset_path / "05-B0/data/fmap"

PI_UNICODE = "\U0001D70B"
fname_mag_e1 = data_dir / "sub-fmap_magnitude1.nii.gz"
fname_phase_e1 = data_dir / "sub-fmap_phase1.nii.gz"
fname_phase_e1_json = data_dir / "sub-fmap_phase1.json"
fname_phase_e2 = data_dir / "sub-fmap_phase2.nii.gz"
fname_mask = data_dir / "mask.nii.gz"
fname_fmap = data_dir / "fmap.nii.gz"

nii_mag_e1 = nib.load(fname_mag_e1)
nii_phase_e1 = nib.load(fname_phase_e1)
nii_phase_e2 = nib.load(fname_phase_e2)
nii_mask = nib.load(fname_mask)
nii_fmap = nib.load(fname_fmap)

# Phase evolution though different echo times
mask = nii_mask.get_fdata()[30:-30,8:105,30]
fmap = nii_fmap.get_fdata()[30:-30,8:105,30] * mask  # [Hz]
phase1 = (nii_phase_e1.get_fdata()[30:-30,8:105,30] / 4095 * 2 * math.pi - math.pi) * mask

with open(fname_phase_e1_json, 'r') as json_data1:
    data1 = json.load(json_data1)
    
echo_time1 = data1['EchoTime']
phase0 = phase1 - (echo_time1 * (fmap * 2 * math.pi))
zmin = -math.pi
zmax = math.pi

steps = 31
last_echo_time = 0.03
echo_times = np.linspace(0.0, last_echo_time, steps)
fig = go.Figure()
for i_echo, echo_time in enumerate(echo_times):
    phase = phase0 + (fmap * echo_time * 2 * math.pi)
    phase = np.angle(np.exp(1j*phase))
    if i_echo >= len(echo_times) - 1:
        fig.add_trace(go.Heatmap(z=np.rot90(phase, k=-1), visible=True, coloraxis = "coloraxis"))
    else:
        fig.add_trace(go.Heatmap(z=np.rot90(phase, k=-1), visible=False, coloraxis = "coloraxis"))

fig.update_layout(coloraxis = {'colorscale':'gray'},
                 coloraxis_cmin=zmin, coloraxis_cmax=zmax)
fig.update_coloraxes(
    colorbar=dict(title="Rad",
                  titleside="top",
                  tickmode="array",
                  tickvals=[-math.pi, 0, math.pi-0.01],
                  ticktext = [f"-{PI_UNICODE}", 0, f'{PI_UNICODE}']))

echo_times_str = [f"{time:.2}" for time in echo_times]
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label=echo_times_str[i],
        args=[{"visible": [False] * len(fig.data)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=30,
    currentvalue={"prefix": "Echo Time (s): "},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.update_layout(
    title=dict(text="Phase at different echo times", x=0.5)
)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.update_layout({"height": 550, "width": 500})
fig.show()